# Semantic search with AlloyDB for PostgreSQL and OpenAI

This notebook walks you through using [Google Cloud AlloyDB](https://cloud.google.com/products/alloydb) as a vector database for OpenAI embeddings. It demonstrates how to:

1. Use and create embeddings from OpenAI API.
2. Store embeddings in an AlloyDB database.
3. Use AlloyDB with the `pgvector` extension to perform vector similarity search.
4. Create and use ScaNN indexes to boost your queries

## Before you begin

To run this notebook, you will need the following:

 * [A Google Cloud Project](https://developers.google.com/workspace/guides/create-project).
 * [An AlloyDB cluster and instance](https://cloud.google.com/alloydb/docs/cluster-create). You can create a free trial cluster, see instructions here [AlloyDB Free Trial Cluster](https://cloud.google.com/alloydb/docs/free-trial-cluster) or use a [local AlloyDB Omni](https://cloud.google.com/alloydb/omni/current/docs/overview).
 * [An OpenAI API key](https://platform.openai.com/account/api-keys).


## Install required modules

This notebook requires several packages that you can install with `pip`:


In [ ]:
!python3 -m pip install -qU openai psycopg2-binary pandas wget

## Set your AlloyDB database values and Open AI API key
Find your database values in the [AlloyDB Instances page](https://console.cloud.google.com/alloydb/clusters) and your OpenAI API key on [your account](https://platform.openai.com/account/api-keys).

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
HOST = "xx.xx.xx.xx" # @param {type: "string"}
PORT = "5432" # @param {type: "string"}
DATABASE = "postgres"  # @param {type: "string"}
USER = "postgres"  # @param {type: "string"}
PASSWORD = "secure-password"  # @param {type: "string"}
OPENAI_KEY = "sk-xxx"  # @param {type: "string"}

## Connect to your AlloyDB instance
Using the values you just set, you can now connect to your AlloyDB for Postgres.

In [ ]:
import psycopg2

# Instantiate connection
connection = psycopg2.connect(
    host=HOST,
    port=PORT,
    database=DATABASE,
    user=USER,
    password=PASSWORD
)
connection.set_session(autocommit=True)

# Create a new cursor object
cursor = connection.cursor()

### Test the connection

In [ ]:
# Execute a simple query to test the connection
cursor.execute("SELECT 'AlloyDB is amazing!';")
result = cursor.fetchone()

# Check the query result
if result == ("AlloyDB is amazing!",):
    print("You are ready to query!")
else:
    print("Connection failed.")

## Set up your AlloyDB cluster for vector search

To store your embeddings and execute similarity search queries, we will use the PostgreSQL `pgvector` extension.

As for the vector indexes, we will use Google ScaNN through the extension `alloydb_scann`. More information on this unique index from Google [can be found here](https://cloud.google.com/blog/products/databases/understanding-the-scann-index-in-alloydb).

In [ ]:
create_extensions_stmt = '''
CREATE EXTENSION IF NOT EXISTS vector;
CREATE EXTENSION IF NOT EXISTS alloydb_scann;
'''

# Execute the SQL statements
cursor.execute(create_extensions_stmt)

Now we will create the table that will store our data and our embeddings, using the `vector` data type.

In [ ]:
create_table_stmt = f'''
DROP TABLE IF EXISTS vector_store;
CREATE TABLE IF NOT EXISTS vector_store (
    id INTEGER NOT NULL,
    url TEXT,
    title TEXT,
    content TEXT,
    title_vector vector(1536),
    content_vector vector(1536),
    vector_id INT
)
'''

# Execute the SQL statements
cursor.execute(create_table_stmt)

And lastly, we create the vector indexes with Google ScaNN.

In [ ]:
create_scann_index_stmt = f'''
CREATE INDEX scann_index_title ON vector_store
  USING scann (title_vector cosine)
  WITH (num_leaves = 200, max_num_levels = 2);

CREATE INDEX scann_index_content ON vector_store
  USING scann (content_vector cosine)
  WITH (num_leaves = 200, max_num_levels = 2);
'''

# Execute the SQL statements
cursor.execute(create_scann_index_stmt)

## Load the data into AlloyDB

In this cookbook, we are gonna use the Simple English Wikipedia dataset hosted by OpenAI, with pre-calculated embeddings.

It contains 25000 rows, import will take few minutes.

In [ ]:
import wget

embeddings_url = "https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip"

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

In [ ]:
import zipfile
import os
import re
import tempfile

current_directory = os.getcwd()
zip_file_path = os.path.join(current_directory, "vector_database_wikipedia_articles_embedded.zip")
output_directory = os.path.join(current_directory, "../../data")

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(output_directory)

# Check to see if the csv file was extracted
file_name = "vector_database_wikipedia_articles_embedded.csv"
data_directory = os.path.join(current_directory, "../../data")
file_path = os.path.join(data_directory, file_name)

if os.path.exists(file_path):
    print(f"The data file {file_name} exists.")
else:
    print(f"The data file {file_name} does not exist.")

We can have a quick look at the data before the ingestion.

In [ ]:
import pandas, json
data = pandas.read_csv('../../data/vector_database_wikipedia_articles_embedded.csv')
data

And now we load the data into AlloyDB.

In [ ]:
import csv

csv_file_path = '../../data/vector_database_wikipedia_articles_embedded.csv'


with open(csv_file_path, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)  # Skip the header row

    copy_command = f"""
        COPY vector_store (id, url, title, content, title_vector, content_vector, vector_id)
        FROM STDIN WITH (FORMAT CSV, HEADER FALSE, DELIMITER ',');
    """

    cursor.copy_expert(copy_command, file)

print(f"Successfully imported data from '{csv_file_path}' into 'vector_store'.")

## Semantic Search with AlloyDB for PostgreSQL

Now that our table is filled with data and embeddings, we are ready to execute similarity search queries.

In this example, we are gonna use the cosine distance to compare our vectors and the embedding of our search string, generated by the OpenAI API.

In [ ]:
from openai import OpenAI

def semantic_search(query, vector_name="title_vector", top_k=20):
    # Creates embedding vector from user query
    client = OpenAI(api_key=OPENAI_KEY)
    embedded_query = client.embeddings.create(
        input=query,
        model="text-embedding-ada-002",
    ).data[0].embedding

    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "[" + ",".join(map(str, embedded_query)) + "]"

    # Create SQL query
    query_sql = f"""
      SELECT id, url, title, cosine_distance({vector_name},'{embedded_query_pg}') AS cosine_distance
      FROM vector_store
      ORDER BY {vector_name} <=> '{embedded_query_pg}'
      LIMIT {top_k};
      """

    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results

In [ ]:
query_results = semantic_search("French history", "content_vector", 20)
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. French Revolution (Score: 0.847)
2. La Marseillaise (Score: 0.832)
3. New France (Score: 0.83)
4. Bastille (Score: 0.829)
5. Restauration (Score: 0.823)
6. Angevin (Score: 0.821)
7. Salic law (Score: 0.82)
8. Fin de siècle (Score: 0.82)
9. Tennis Court Oath (Score: 0.82)
10. History of Spain (Score: 0.818)
11. Gaul (Score: 0.817)
12. Dreyfus Affair (Score: 0.816)
13. Declaration of the Rights of Man and of the Citizen (Score: 0.815)
14. Siege of Orleans (Score: 0.814)
15. Palace of Versailles (Score: 0.814)
16. Franco-Prussian War (Score: 0.811)
17. Arc de Triomphe (Score: 0.811)
18. Louise Michel Battalions (Score: 0.808)
19. Treaty of Verdun (Score: 0.808)
20. Bastide (Score: 0.808)